In [1]:
from melee_dataset import MeleeDataset
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from torch.distributions import Categorical
from torch.distributions import Bernoulli,Normal
import matplotlib.pyplot as plt
from PolicyNet import PolicyNet
import torch.nn.functional as F
from Agents.BCAgent import BCAgent

%load_ext autoreload
%autoreload 2

In [2]:
# train_dataset = MeleeDataset(data_path="data/train_mini_515")
train_dataset = MeleeDataset(data_path="data/train_mini_100")
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [3]:
print(f"obs_dim: {train_dataset[0][0].shape}")
print(f"act_dim: {train_dataset[0][1].shape}")

Loaded Game_20250501T001638.slp with 15927 states and 15927 actions. The cache size is 1.
obs_dim: torch.Size([70])
act_dim: torch.Size([17])


In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
obs_dim = train_dataset[0][0].shape[0]
act_dim = train_dataset[0][1].shape[0]
policy  = PolicyNet(obs_dim, act_dim).to(device)
# opt     = optim.Adam(policy.parameters(), lr=1e-3)
agent = BCAgent(obs_dim, act_dim, device)

Using device: cuda


In [5]:
losses = []
for epoch in range(6):
    total_loss = 0.0
    for i, (states, actions, next_states) in tqdm(enumerate(train_loader)):
        states = states.to(device)       # [B, obs_dim]
        actions = actions.to(device)     # [B] integers in [0…act_dim-1]
        next_states = next_states.to(device)

        loss = agent.train(states, actions, next_states)

        total_loss += loss * states.shape[0]
        losses.append(loss)
    avg_loss = total_loss / len(train_dataset)
    print(f"Epoch {epoch+1} — Loss: {avg_loss:.4f}")

0it [00:00, ?it/s]

Loaded Game_20250501T001214.slp with 15565 states and 15565 actions. The cache size is 2.
Loaded Game_20250430T154348.slp with 8096 states and 8096 actions. The cache size is 3.
Loaded Game_20250430T051051.slp with 15510 states and 15510 actions. The cache size is 4.
Loaded Game_20250430T011027.slp with 9799 states and 9799 actions. The cache size is 5.
Loaded Game_20250430T063350.slp with 10702 states and 10702 actions. The cache size is 6.
Loaded Game_20250430T134932.slp with 14271 states and 14271 actions. The cache size is 7.
Loaded Game_20250430T114640.slp with 7386 states and 7386 actions. The cache size is 8.
Loaded Game_20250430T121612.slp with 12745 states and 12745 actions. The cache size is 9.
Loaded Game_20250430T145017.slp with 6138 states and 6138 actions. The cache size is 10.
Loaded Game_20250430T065555.slp with 14492 states and 14492 actions. The cache size is 11.
Loaded Game_20250430T234610.slp with 7291 states and 7291 actions. The cache size is 12.
Loaded Game_20250

0it [00:39, ?it/s]


KeyboardInterrupt: 

In [16]:
# Save the trained model
torch.save(policy.state_dict(), "bc_100_train.pth")
print("Model saved to trained_policy.pth")


Model saved to trained_policy.pth


In [17]:
agent.predict(train_dataset[100][0].to(device))

array([-2.1051790e-06,  1.1450386e-05,  3.7896403e-05, -1.8594084e-05,
       -2.7943970e-05,  9.6808681e-06,  3.3912274e-06,  3.0563529e-05,
        1.3602929e-05,  4.0085633e-05, -6.3856984e-05,  3.8956523e-01,
        4.2213976e-01,  5.0000000e-01,  5.0000000e-01,  7.3807001e-02,
        3.8145036e-01], dtype=float32)